# Chicago food inspections project

## Exploratory data analysis

### Prepare data science tools

In [1]:
import numpy as np
import pandas as pd

In [2]:
import dask.dataframe as dd

In [21]:
import pandas_profiling

In [39]:
import plotly.express as px

### Read data

We're first trying to read data from the Food Inspections dataset - https://data.cityofchicago.org/Health-Human-Services/Food-Inspections/4ijn-s7e5/data - any supplementary data data will be read later on (e.g. weather conditions or socio-economical data)

In [11]:
df = dd.read_csv('Food_Inspections_general_201911.csv', 
                 dtype={'License #': 'Int64',
                        'Zip': 'Int64'})

# The lack of NaN rep in integer columns is a pandas "gotcha", 
# but it's fixable with a new type Int64Dtype

### List of columns

In [12]:
df.columns

Index(['Inspection ID', 'DBA Name', 'AKA Name', 'License #', 'Facility Type',
       'Risk', 'Address', 'City', 'State', 'Zip', 'Inspection Date',
       'Inspection Type', 'Results', 'Violations', 'Latitude', 'Longitude',
       'Location'],
      dtype='object')

### View first rows

First glance at the data. Surprisingly, most of the data is un-anonimized, meaning it's possible to identify the exact restaurant which was audited. 

In [13]:
df.head(5)

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2345699,JET'S PIZZA,JET'S PIZZA,2163956,Restaurant,Risk 2 (Medium),2811 N ASHLAND AVE,CHICAGO,IL,60657,11/15/2019,Canvass,Pass w/ Conditions,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.932766,-87.668262,"(-87.66826200882875, 41.93276573571165)"
1,2345619,JAIPUR,JAIPUR,2694084,Restaurant,Risk 1 (High),738 W RANDOLPH ST,CHICAGO,IL,60661,11/14/2019,Canvass,Pass,NaN,41.884518,-87.647304,"(-87.64730383120978, 41.88451799637232)"
2,2345616,VIP FIT CLUB,VIP FIT CLUB,2446547,Restaurant,Risk 2 (Medium),3426 W DIVERSEY AVE,CHICAGO,IL,60647,11/14/2019,Complaint,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.932069,-87.713294,"(-87.71329441237397, 41.932068626464286)"
3,2345602,VELVET TACO,VELVET TACO,2652941,Restaurant,Risk 3 (Low),2309 N LINCOLN AVE,CHICAGO,IL,60614,11/14/2019,License,Fail,NaN,41.923953,-87.646462,"(-87.6464616293504, 41.9239529807269)"
4,2345603,VELVET TACO,VELVET TACO,2652943,Restaurant,Risk 3 (Low),2309 N LINCOLN AVE,CHICAGO,IL,60614,11/14/2019,License,Fail,NaN,41.923953,-87.646462,"(-87.6464616293504, 41.9239529807269)"


Some data is unstructured, especially in text columns like 'Violations', which leaves an opportunity to use NLP tools (for gathering additional information from text). Next, this data has an (inspection) date column, which makes for some time-series analysis.

### Get data types of columns

In [14]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 17 entries, Inspection ID to Location
dtypes: Int64(2), object(12), float64(2), int64(1)

Many columns are text (represented by the 'object' type)

In [18]:
df.dtypes

Inspection ID        int64
DBA Name            object
AKA Name            object
License #            Int64
Facility Type       object
Risk                object
Address             object
City                object
State               object
Zip                  Int64
Inspection Date     object
Inspection Type     object
Results             object
Violations          object
Latitude           float64
Longitude          float64
Location            object
dtype: object

### Number of rows

In [20]:
f'There are {len(df)} rows in dataset and {len(df.columns)} columns'

'There are 195979 rows in dataset and 17 columns'

### Basic reconnaissance analysis

#### Use random sample

In [25]:
df.sample(frac=0.05).compute().profile_report(style={'full_width':True})

### Basic cleaning and checks

In [29]:
df.City.value_counts().compute()

CHICAGO               195139
Chicago                  321
chicago                   97
CCHICAGO                  46
SCHAUMBURG                25
                       ...  
COUNTRY CLUB HILLS         1
DES PLAINES                1
GRIFFITH                   1
alsip                      1
GLENCOE                    1
Name: City, Length: 71, dtype: int64

In [31]:
# TODO: normalize city names

In [30]:
df.State.value_counts().compute()

IL    195934
WI         1
NY         1
IN         1
Name: State, dtype: int64

In [32]:
# State other than 'IL' is probably a noice and it'll be removed

### Exploratory analysis of time-series data (part I)

In [37]:
inspections_through_time = df[['Inspection Date', 'Inspection Type', 'Results']].compute()

In [38]:
inspections_through_time

,Inspection Date,Inspection Type,Results
0,11/15/2019,Canvass,Pass w/ Conditions
1,11/14/2019,Canvass,Pass
2,11/14/2019,Complaint,Fail
3,11/14/2019,License,Fail
4,11/14/2019,License,Fail
...,...,...,...
22506,03/02/2010,Complaint,Fail
22507,02/22/2010,Complaint,Pass
22508,01/13/2010,Canvass,Pass
22509,03/11/2010,Complaint,Pass


In [46]:
inspections_through_time_aggr = inspections_through_time.groupby(['Inspection Type', 
                                                                  'Results']).size().reset_index(name='counts')

In [58]:
inspections_through_time_aggr.sort_values('counts', ascending=False)

,Inspection Type,Results,counts
19,Canvass,Pass,48795
15,Canvass,Fail,17668
18,Canvass,Out of Business,16586
25,Canvass Re-Inspection,Pass,16397
20,Canvass,Pass w/ Conditions,15788
...,...,...,...
78,LICENSE WRONG ADDRESS,Fail,1
77,LICENSE TASK FORCE / NOT -FOR-PROFIT CLUB,Pass,1
76,LICENSE TASK FORCE / NOT -FOR-PROFIT CLU,Pass,1
73,LICENSE RENEWAL INSPECTION FOR DAYCARE,Pass,1


In [59]:
fig = px.bar(inspections_through_time_aggr[inspections_through_time_aggr['Inspection Type'].isin(['Canvass', 'Complaint', 'License'])], 
             x='Inspection Type', y="counts", 
             color='Results', barmode='stack')
fig.show()